In [1]:
import h5py
import numpy as np
import glob
from shutil import copyfile

import matplotlib.pyplot as plt

%matplotlib inline
#%matplotlib notebook

In [178]:
# Node class

class Node:

    def __init__ (self, filename):

        self.filename = filename

        # Read data from file

        f = h5py.File(filename, 'r')

        self.Teff = f.attrs['Teff']
        self.logg = f.attrs['logg']

        self.data = f['data'][...]

        f.close()

# Grid class

class Grid:

    def __init__ (self, filenames):

        # Read nodes

        nodes_list = np.empty_like(filenames, dtype=object)

        for i, filename in enumerate(filenames):
            nodes_list[i] = Node(filename)

        # Extract axes

        self.Teff_axis = np.unique([node.Teff for node in nodes_list])
        self.logg_axis = np.unique([node.logg for node in nodes_list])

        self.n_Teff = len(self.Teff_axis)
        self.n_logg = len(self.logg_axis)

        # Set up the 2-D (Teff,logg) node array

        self.nodes = np.empty([self.n_Teff,self.n_logg], dtype=object)

        for node in nodes_list:

            # Find where the node belongs in the list

            i_Teff = np.where(node.Teff == self.Teff_axis)[0][0]
            i_logg = np.where(node.logg == self.logg_axis)[0][0]

            self.nodes[i_Teff,i_logg] = node

        # Set up the 2-D (Teff,logg) neighbors array
        
        self.neighbors = np.empty((self.n_Teff,self.n_logg), dtype=np.ndarray)
        
        for i_logg in range(self.n_logg):
            
            for i_Teff in range(self.n_Teff):
                
                # Identify neighbors to node
                
                self.neighbors[i_Teff,i_logg] = self.find_neighbors((i_Teff,i_logg))
            

            
    def lookup (self, Teff, logg):

        if Teff in self.Teff_axis and logg in self.logg_axis:

            # Find location in array

            i_Teff = np.where(Teff == self.Teff_axis)[0][0]
            i_logg = np.where(logg == self.logg_axis)[0][0]

            # Return the node if it exists

            node = self.nodes[i_Teff,i_logg]
            
            if isinstance(node, Node):
                return node
            else:
                return None

        else:

            return None
        

    def show_topology (self):

        for i_logg in range(self.n_logg):

            for i_Teff in range(self.n_Teff):

                if isinstance(self.nodes[i_Teff,i_logg], Node):
                    print("X", end=' ')
                else:
                    print(".", end=' ')

            print()
            print()
            
            
    def locate (self, Teff, logg):
        
        i_0, i_1 = 0, self.n_Teff-1
        j_0, j_1 = 0, self.n_logg-1
        
        # First check Teff, logg within grid
        
        if (Teff < self.Teff_axis[i_0]) or (Teff > self.Teff_axis[i_1]):
            raise IndexError(f'Teff={Teff} outside axis range')
        if (logg < self.logg_axis[j_0]) or (logg > self.logg_axis[j_1]):
            raise IndexError(f'logg={logg} outside axis range')
        
        # Find i such that Teff_axis[i] <= Teff < Teff_axis[i+1]
        # using bisection method

        while (i_1 - i_0) > 1:
    
            i_m = (i_0 + i_1)//2
        
            if Teff >= self.Teff_axis[i_m]:
                i_0 = i_m
            elif Teff < self.Teff_axis[i_m]:
                i_1 = i_m
            
        # check special case: Teff == max(Teff_axis)
        
        if Teff == self.Teff_axis[i_1]:
            i = i_1 - 1
        else:
            i = i_m
            
        # Find j such that logg_axis[j] <= logg < logg_axis[j+1]
        # using bisection method

        while (j_1 - j_0) > 1:
    
            j_m = (j_0 + j_1)//2
        
            if logg >= self.logg_axis[j_m]:
                j_0 = j_m
            elif logg < self.logg_axis[j_m]:
                j_1 = j_m
            
        # check special case: logg == max(logg_axis)
        
        if logg == self.logg_axis[j_1]:
            j = j_1 - 1
        else:
            j = j_m
            
        # Return tuple (i,j)
        print(i,j)
        print(self.Teff_axis[i], self.logg_axis[j])
        return (i,j)
    
    
    def find_neighbors (self, ij, show=False):
        
        stencil = np.zeros((3,3), dtype=bool)
        
        # Find where node has neighbors
        
        for k in range(-1,2):
            
            for h in range(-1,2):
                
                j_Teff = ij[0] + h
                j_logg = ij[1] + k
                
                # if neighbor exists, store True in bool stencil
                
                if ((0 <= j_Teff) and (j_Teff < self.n_Teff)) and \
                ((0 <= j_logg) and (j_logg < self.n_logg)):
                
                    if isinstance(self.nodes[j_Teff,j_logg], Node):
                        if show==True: print("X", end=' ')
                        stencil[h+1, k+1] = True
                    else:
                        if show==True: print(".", end=' ')
                    
            if show==True:    
                print()
                print()
        
        return stencil
        

In [169]:
grid = Grid(glob.glob('*h5'))

In [172]:
try:
    #user_Teff = input('Enter Teff: ')
    #user_logg = input('Enter logg: ')
    
    loc = grid.locate(30000, 3.0)
    grid.show_topology()
    loc_ns = grid.neighbors
    
except IndexError:
    print('try a different (Teff,logg)!')
    raise

8 1
40000.0 3.0
X X X X X X . . . . 

X X X X X X X X . . 

X X X X X X X X X . 

X X X X X X X X X X 

X X X X X X X X X X 

